# GameTracker Processing Engine

**One-time setup:** Run cells 1-3 once ever.
**Every match day:** Runtime -> Run All, leave this tab open.

Cell 5 polls GameTracker/jobs/ every 30 s and processes jobs automatically.
No URLs, no tokens, no ngrok.

---
Before running: Runtime -> Change runtime type -> T4 GPU

In [ ]:
# CELL 1 - Install dependencies (~90 seconds, every session)
!pip install ultralytics easyocr filterpy scipy -q
!apt-get install -y ffmpeg libsm6 libxext6 -q

import torch
gpu = torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'NOT FOUND'
print(f'GPU: {gpu}')
print('Dependencies installed')

In [ ]:
# CELL 2 - Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
BASE   = '/content/drive/MyDrive/GameTracker'
JOBS   = os.path.join(BASE, 'jobs')
RAW    = os.path.join(BASE, 'raw')
OUTPUT = os.path.join(BASE, 'output')
MODELS = os.path.join(BASE, 'models')
WORK   = '/content/gametracker_work'

for d in [JOBS, RAW, OUTPUT, MODELS, WORK]:
    os.makedirs(d, exist_ok=True)

print(f'Drive mounted. Jobs: {JOBS}')

In [ ]:
# CELL 3 - Download YOLO model (once ever, ~50 MB)
import shutil
from ultralytics import YOLO

model_path = os.path.join(MODELS, 'yolov8m.pt')
if not os.path.exists(model_path):
    print('Downloading YOLOv8m...')
    YOLO('yolov8m.pt')
    shutil.copy('yolov8m.pt', model_path)
    print('Saved to Drive')
else:
    print('Model already in Drive')

import numpy as np, cv2
_m = YOLO(model_path)
_m(np.zeros((640,640,3), dtype=np.uint8), verbose=False)
del _m
print('GPU warmed up - ready')

In [ ]:
# CELL 4 - All pipeline functions (fully implemented)
import json, time, os, subprocess, collections
import cv2
import numpy as np
from datetime import datetime
from ultralytics import YOLO
import easyocr
from filterpy.kalman import KalmanFilter
import scipy.io.wavfile as wav
from scipy.signal import correlate


# ── Helpers ──────────────────────────────────────────────────────────────────

def write_status(jobs_dir, job_id, stage_index, stage_name,
                 progress, message='', status='running', extra=None):
    payload = {
        'job_id':      job_id,
        'status':      status,
        'stage_index': stage_index,
        'stage_total': 7,
        'stage':       stage_name,
        'progress':    int(progress),
        'message':     message,
        'updated':     datetime.utcnow().isoformat(),
    }
    if extra:
        payload.update(extra)
    with open(os.path.join(jobs_dir, f'status_{job_id}.json'), 'w') as f:
        json.dump(payload, f)

def write_error(jobs_dir, job_id, msg):
    write_status(jobs_dir, job_id, -1, 'Error', 0, msg, status='error')

def hex_to_bgr(h):
    h = h.lstrip('#')
    return (int(h[4:6], 16), int(h[2:4], 16), int(h[0:2], 16))

def make_kalman():
    kf = KalmanFilter(dim_x=4, dim_z=2)
    kf.F = np.array([[1,0,1,0],[0,1,0,1],[0,0,1,0],[0,0,0,1]], dtype=np.float32)
    kf.H = np.array([[1,0,0,0],[0,1,0,0]], dtype=np.float32)
    kf.P *= 500; kf.R *= 5; kf.Q *= 0.1
    return kf


# ── Stage 1: Audio sync ───────────────────────────────────────────────────────

def sync_videos(cam_a, cam_b, jobs_dir, job_id):
    """
    Cross-correlate audio from both cameras to find the time offset
    introduced by the clap sync marker.
    Returns offset_seconds: positive = cam_b starts later.
    """
    write_status(jobs_dir, job_id, 0, 'Audio Sync', 10, 'Extracting audio...')

    def to_wav(src, dst):
        subprocess.run([
            'ffmpeg', '-i', src, '-vn', '-acodec', 'pcm_s16le',
            '-ar', '22050', '-ac', '1', dst, '-y', '-loglevel', 'error'
        ], check=True)

    wa = os.path.join(WORK, 'sync_a.wav')
    wb = os.path.join(WORK, 'sync_b.wav')
    to_wav(cam_a, wa); to_wav(cam_b, wb)

    write_status(jobs_dir, job_id, 0, 'Audio Sync', 60, 'Cross-correlating...')
    sr_a, a = wav.read(wa); a = a.astype(np.float32)
    sr_b, b = wav.read(wb); b = b.astype(np.float32)

    n = min(sr_a * 60, len(a), len(b))
    a = a[:n] / (np.abs(a[:n]).max() + 1e-9)
    b = b[:n] / (np.abs(b[:n]).max() + 1e-9)
    lag    = np.argmax(correlate(a, b, mode='full')) - (n - 1)
    offset = lag / sr_a

    write_status(jobs_dir, job_id, 0, 'Audio Sync', 100, f'Offset: {offset:+.3f}s')
    print(f'  Sync offset: {offset:+.3f}s')
    return offset


# ── Stage 2: Lens correction ──────────────────────────────────────────────────

def build_lens_maps(shape):
    """Barrel distortion maps for a generic wide-angle action camera."""
    h, w = shape[:2]
    K = np.array([[w*0.72, 0, w/2],[0, w*0.72, h/2],[0, 0, 1]], dtype=np.float64)
    D = np.array([-0.30, 0.10, 0.0, 0.0], dtype=np.float64)
    nK = cv2.fisheye.estimateNewCameraMatrixForUndistortRectify(
        K, D, (w, h), np.eye(3), balance=0.6)
    m1, m2 = cv2.fisheye.initUndistortRectifyMap(
        K, D, np.eye(3), nK, (w, h), cv2.CV_16SC2)
    return m1, m2

def undistort(frame, m1, m2):
    return cv2.remap(frame, m1, m2, cv2.INTER_LINEAR)


# ── Stage 3: Panorama stitch ──────────────────────────────────────────────────

def compute_homography(fa, fb, overlap_pct):
    """
    SIFT feature matching in the overlap zone.
    Returns H (3x3) that warps fb onto fa's coordinate system.
    """
    h, w = fa.shape[:2]
    opx  = int(w * overlap_pct / 100)
    ga   = cv2.cvtColor(fa[:, w-opx:], cv2.COLOR_BGR2GRAY)
    gb   = cv2.cvtColor(fb[:, :opx],   cv2.COLOR_BGR2GRAY)

    sift = cv2.SIFT_create(nfeatures=3000)
    kpa, desa = sift.detectAndCompute(ga, None)
    kpb, desb = sift.detectAndCompute(gb, None)

    if desa is None or desb is None or len(kpa) < 10 or len(kpb) < 10:
        raise RuntimeError(
            f'Too few SIFT features ({len(kpa or [])}/{len(kpb or [])}) in overlap zone. '
            'Increase overlap % or check camera alignment.')

    flann   = cv2.FlannBasedMatcher({'algorithm': 1, 'trees': 5}, {'checks': 80})
    matches = flann.knnMatch(desa, desb, k=2)
    good    = [m for m, n in matches if m.distance < 0.72 * n.distance]

    if len(good) < 10:
        raise RuntimeError(f'Only {len(good)} good matches (need >=10). '
                           'Is there enough pitch overlap at centre circle?')

    pta = np.float32([kpa[m.queryIdx].pt for m in good])
    ptb = np.float32([kpb[m.trainIdx].pt for m in good])
    pta[:, 0] += w - opx   # shift to full-frame coords

    H, mask = cv2.findHomography(ptb, pta, cv2.RANSAC, 5.0)
    print(f'  Homography: {len(good)} matches, {mask.sum() if mask is not None else 0} inliers')
    return H

def stitch_frame(fa, fb, H, overlap_pct, canvas_w):
    """Warp fb onto fa and linearly blend the seam zone."""
    h, w = fa.shape[:2]
    opx  = int(w * overlap_pct / 100)
    canvas = np.empty((h, canvas_w, 3), dtype=np.uint8)
    canvas[:, :w] = fa
    warped = cv2.warpPerspective(fb, H, (canvas_w, h))
    canvas[:, w:] = warped[:, w:]
    alpha = np.linspace(0, 1, opx, dtype=np.float32)[None, :, None]
    seam  = w - opx
    canvas[:, seam:w] = (
        (1 - alpha) * fa[:, seam:w].astype(np.float32) +
        alpha       * warped[:, seam:w].astype(np.float32)
    ).astype(np.uint8)
    return canvas


# ── Drawing helpers ───────────────────────────────────────────────────────────

def draw_name_tag(frame, cx, top_y, label, bgr):
    """Pill label with connector line above a player bounding box."""
    font  = cv2.FONT_HERSHEY_DUPLEX
    scale = 0.42; thick = 1
    (tw, th), _ = cv2.getTextSize(label, font, scale, thick)
    pad = 4
    x1  = cx - tw // 2 - pad
    y2  = max(top_y - 6, th + pad * 2)
    y1  = y2 - th - pad * 2
    x2  = x1 + tw + pad * 2
    cv2.rectangle(frame, (x1, y1), (x2, y2), bgr, -1)
    cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 255, 255), 1)
    cv2.line(frame, (cx, y2), (cx, top_y), (200, 200, 200), 1)
    cv2.putText(frame, label, (x1 + pad, y2 - pad),
                font, scale, (255, 255, 255), thick, cv2.LINE_AA)

def draw_ball_trail(frame, trail, fw, fh):
    """Fading trail of the last N ball positions."""
    pts = list(trail)
    for i in range(1, len(pts)):
        alpha = i / len(pts)
        col   = (0, int(120 * alpha), int(255 * alpha))
        t     = max(1, int(alpha * 4))
        x1, y1 = pts[i-1]; x2, y2 = pts[i]
        if 0<=x1<fw and 0<=y1<fh and 0<=x2<fw and 0<=y2<fh:
            cv2.line(frame, (x1, y1), (x2, y2), col, t, cv2.LINE_AA)

def draw_score_bug(frame, home_name, away_name, home_score, away_score,
                   home_bgr, away_bgr, elapsed_secs):
    """TV-style score bug in top-left corner."""
    font  = cv2.FONT_HERSHEY_DUPLEX
    small = 0.5; bold = 1
    bar_h = 28; pad = 8
    mins  = int(elapsed_secs // 60)
    secs  = int(elapsed_secs % 60)
    ov = frame.copy()
    cv2.rectangle(ov, (10, 10), (310, 10 + bar_h*3 + pad), (20,20,20), -1)
    cv2.addWeighted(ov, 0.70, frame, 0.30, 0, frame)
    # Home row
    cv2.rectangle(frame, (10, 10), (20, 10+bar_h), home_bgr, -1)
    cv2.putText(frame, home_name[:12].upper(), (25, 10+bar_h-8),
                font, small, (255,255,255), bold, cv2.LINE_AA)
    cv2.putText(frame, str(home_score), (280, 10+bar_h-8),
                font, small, (255,255,255), bold, cv2.LINE_AA)
    # Away row
    cv2.rectangle(frame, (10, 10+bar_h), (20, 10+bar_h*2), away_bgr, -1)
    cv2.putText(frame, away_name[:12].upper(), (25, 10+bar_h*2-8),
                font, small, (255,255,255), bold, cv2.LINE_AA)
    cv2.putText(frame, str(away_score), (280, 10+bar_h*2-8),
                font, small, (255,255,255), bold, cv2.LINE_AA)
    # Timer
    cv2.putText(frame, f'{mins:02d}:{secs:02d}', (130, 10+bar_h*3-4),
                font, 0.45, (180,180,180), 1, cv2.LINE_AA)


# ── Stage 5: Goal detection ───────────────────────────────────────────────────

def estimate_goal_boxes(canvas_w, h):
    """
    Estimate goal mouth bounding boxes in panorama coordinates.
    Goals are at far left and far right, vertically centred.
    """
    mid_y1 = int(h * 0.35); mid_y2 = int(h * 0.65)
    gw     = int(canvas_w * 0.04)
    return {
        'left':  (0,            mid_y1, gw,          mid_y2),
        'right': (canvas_w-gw,  mid_y1, canvas_w,    mid_y2),
    }

def detect_goals(ball_positions, goal_boxes, fps, conf_threshold):
    """
    Scan ball_positions for sequences where the ball stays inside a
    goal bounding box for enough consecutive frames.

    conf_threshold 50-95 controls how many frames are required.
    Returns list of {'frame', 'side', 'time_secs'}.
    """
    if not ball_positions:
        return []
    # Higher confidence = more frames needed before flagging
    required = max(3, int(3 * conf_threshold / 60))
    events   = []
    runs     = {s: 0 for s in goal_boxes}
    flagged  = {s: False for s in goal_boxes}

    for (fidx, bx, by) in ball_positions:
        for side, (gx1, gy1, gx2, gy2) in goal_boxes.items():
            if gx1 <= bx <= gx2 and gy1 <= by <= gy2:
                runs[side] += 1
                if runs[side] >= required and not flagged[side]:
                    events.append({'frame': fidx, 'side': side,
                                   'time_secs': fidx / fps})
                    flagged[side] = True
            else:
                runs[side] = 0
                flagged[side] = False   # allow re-detection after ball leaves

    return events


# ── Stage 7: Ball-following virtual camera ────────────────────────────────────

def smooth_positions(positions, window=15):
    """Moving-average smooth a list of (x,y) tuples."""
    if len(positions) < 2:
        return positions
    xs = np.array([p[0] for p in positions], dtype=np.float32)
    ys = np.array([p[1] for p in positions], dtype=np.float32)
    k  = np.ones(window) / window
    return list(zip(np.convolve(xs, k, 'same').astype(int),
                    np.convolve(ys, k, 'same').astype(int)))

def render_ball_following(pano_path, ball_positions, canvas_w, h, fps,
                          out_path, job, goals,
                          home_name, away_name, home_bgr, away_bgr,
                          jobs_dir, job_id):
    """
    Read the annotated panorama, apply a spring-physics virtual camera
    centred on the ball, composite all overlays, and encode the final MP4.

    Camera behaviour:
    - Spring attraction to ball position (smooth lag, no jerkiness)
    - Zoom out when ball moves fast, zoom in during slow play
    - Falls back to pitch centre when ball is lost
    - Score bug, match timer, ball trail, and goal banner all composited here
    """
    write_status(jobs_dir, job_id, 6, 'Video Render & Overlays', 5,
                 'Smoothing camera path...')

    ov         = job['output']['overlays']
    show_trail = ov.get('ball_trail', True)
    show_score = ov.get('score',      True)
    show_timer = ov.get('timer',      True)
    show_goal  = ov.get('goal_alert', False)

    res_str   = job['output'].get('resolution', '1920')
    out_w, out_h = (2560, 1440) if '1440' in res_str else \
                   (3840, 2160) if '4K'   in res_str else (1920, 1080)
    fps_out   = int(job['output'].get('fps', '60 fps').split()[0])

    # Build frame-indexed lookups
    ball_map   = {f: (bx, by) for f, bx, by in ball_positions}
    raw_xy     = [ball_map[f] for f in sorted(ball_map)]
    smooth_xy  = smooth_positions(raw_xy, window=max(3, fps // 2))
    smooth_map = dict(zip(sorted(ball_map), smooth_xy))

    # Build goal banner frame range
    goal_banner = {}  # frame_idx -> goal event dict
    for ev in goals:
        for i in range(int(fps * 4)):
            goal_banner[ev['frame'] + i] = ev

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    tmp    = out_path.replace('.mp4', '_raw.mp4')
    writer = cv2.VideoWriter(tmp, fourcc, fps_out, (out_w, out_h))

    cap   = cv2.VideoCapture(pano_path)
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cx_pitch = canvas_w // 2
    cy_pitch = h // 2

    # Spring-physics camera state
    cam_x, cam_y = float(cx_pitch), float(cy_pitch)
    vx, vy       = 0.0, 0.0
    SPRING  = 0.08
    DAMPING = 0.75

    trail       = collections.deque(maxlen=18)
    home_score  = 0
    away_score  = 0
    goals_seen  = set()
    fidx        = 0
    last_status = 0

    while True:
        ret, pano = cap.read()
        if not ret:
            break

        # Score updates
        for ev in goals:
            if ev['frame'] == fidx and fidx not in goals_seen:
                goals_seen.add(fidx)
                if ev['side'] == 'right':
                    home_score += 1
                else:
                    away_score += 1

        # Ball target
        if fidx in smooth_map:
            tx, ty = smooth_map[fidx]
            if fidx > 0 and (fidx - 1) in smooth_map:
                px, py = smooth_map[fidx - 1]
                speed  = float(np.hypot(tx - px, ty - py))
            else:
                speed = 0.0
        else:
            tx, ty = cx_pitch, cy_pitch
            speed  = 0.0

        # Spring physics
        vx = (vx + SPRING * (tx - cam_x)) * DAMPING
        vy = (vy + SPRING * (ty - cam_y)) * DAMPING
        cam_x += vx; cam_y += vy

        # Dynamic zoom: wider when ball moves fast
        zoom  = max(0.55, min(0.85, 0.75 - speed * 0.004))
        vp_w  = int(canvas_w * zoom)
        vp_h  = int(h        * zoom)
        cx_c  = int(np.clip(cam_x, vp_w // 2, canvas_w - vp_w // 2))
        cy_c  = int(np.clip(cam_y, vp_h // 2, h         - vp_h // 2))
        x1, x2 = cx_c - vp_w // 2, cx_c + vp_w // 2
        y1, y2 = cy_c - vp_h // 2, cy_c + vp_h // 2

        crop = pano[y1:y2, x1:x2]
        if crop.size == 0:
            crop = pano
        out_frame = cv2.resize(crop, (out_w, out_h), interpolation=cv2.INTER_LINEAR)

        # Ball trail
        if show_trail and fidx in smooth_map:
            bx_s, by_s = smooth_map[fidx]
            sx = int((bx_s - x1) / vp_w * out_w)
            sy = int((by_s - y1) / vp_h * out_h)
            trail.append((sx, sy))
            draw_ball_trail(out_frame, trail, out_w, out_h)
        else:
            trail.clear()

        # Score bug
        if show_score or show_timer:
            draw_score_bug(out_frame, home_name, away_name,
                           home_score, away_score,
                           home_bgr, away_bgr,
                           fidx / fps if show_timer else 0)

        # Goal banner
        if show_goal and fidx in goal_banner:
            ev     = goal_banner[fidx]
            scorer = home_name if ev['side'] == 'right' else away_name
            banner = f'  GOAL!  {scorer.upper()}  '
            (bw, bh), _ = cv2.getTextSize(banner, cv2.FONT_HERSHEY_DUPLEX, 1.4, 2)
            bx_ = (out_w - bw) // 2
            by_ = out_h // 2 - 30
            cv2.rectangle(out_frame, (bx_-10, by_-bh-10),
                          (bx_+bw+10, by_+10), (0, 180, 0), -1)
            cv2.putText(out_frame, banner, (bx_, by_),
                        cv2.FONT_HERSHEY_DUPLEX, 1.4,
                        (255,255,255), 2, cv2.LINE_AA)

        writer.write(out_frame)
        fidx += 1

        if fidx - last_status >= 150:
            pct = min(99, int(fidx / total * 100)) if total else 50
            write_status(jobs_dir, job_id, 6, 'Video Render & Overlays', pct,
                         f'Frame {fidx}/{total}')
            last_status = fidx

    cap.release(); writer.release()

    # Re-encode for compatibility and smaller file size
    subprocess.run([
        'ffmpeg', '-i', tmp,
        '-c:v', 'libx264', '-preset', 'fast', '-crf', '22',
        '-r', str(fps_out), '-movflags', '+faststart',
        out_path, '-y', '-loglevel', 'error'
    ], check=True)
    os.remove(tmp)


# ── Top-level job runner ──────────────────────────────────────────────────────

def run_job(job, jobs_dir, output_dir, work_dir, model_path):
    job_id = job['job_id']
    m      = job['match']
    sc     = job['stitch']
    tc     = job['tracking']
    ov_cfg = job['output']['overlays']

    home_name   = m['home_name'] or 'Home'
    away_name   = m['away_name'] or 'Away'
    home_bgr    = hex_to_bgr(m['home_colour'])
    away_bgr    = hex_to_bgr(m['away_colour'])
    squad_home  = job['squad']['home']
    squad_away  = job['squad']['away']
    shirt_min   = tc['shirt_min']
    shirt_max   = tc['shirt_max']
    overlap_pct = sc['overlap_pct']
    goal_conf   = tc['goal_conf']
    kalman_secs = tc['kalman_window']
    show_names  = ov_cfg.get('names',   True)
    show_nums   = ov_cfg.get('numbers', True)

    cam_a = os.path.join(RAW, job['files']['cam_a'])
    cam_b = os.path.join(RAW, job['files']['cam_b'])
    if not os.path.exists(cam_a): raise FileNotFoundError(f'Camera A not found: {cam_a}')
    if not os.path.exists(cam_b): raise FileNotFoundError(f'Camera B not found: {cam_b}')

    pano_path  = os.path.join(work_dir,   f'{job_id}_pano.mp4')
    final_path = os.path.join(output_dir, f'{job_id}_gametracker.mp4')

    # Stage 1 - Audio sync
    print('\n  [1/7] Audio sync...')
    offset = sync_videos(cam_a, cam_b, jobs_dir, job_id)

    # Stage 2 - Lens maps
    print('  [2/7] Building lens correction maps...')
    write_status(jobs_dir, job_id, 1, 'Lens Distortion Correction', 10, 'Reading frames...')
    def first_frame(path):
        c = cv2.VideoCapture(path); ok, f = c.read(); c.release()
        if not ok: raise RuntimeError(f'Cannot read {path}')
        return f
    m1a, m2a = build_lens_maps(first_frame(cam_a).shape)
    m1b, m2b = build_lens_maps(first_frame(cam_b).shape)
    write_status(jobs_dir, job_id, 1, 'Lens Distortion Correction', 100, 'Done')

    # Stage 3 - Homography
    print('  [3/7] Computing stitch homography...')
    write_status(jobs_dir, job_id, 2, 'Panorama Stitching', 10, 'Matching pitch features...')
    fa0 = undistort(first_frame(cam_a), m1a, m2a)
    fb0 = undistort(first_frame(cam_b), m1b, m2b)
    H   = compute_homography(fa0, fb0, overlap_pct)

    # Open captures with sync offset
    cap_a = cv2.VideoCapture(cam_a)
    cap_b = cv2.VideoCapture(cam_b)
    fps   = cap_a.get(cv2.CAP_PROP_FPS) or 30.0
    hh    = int(cap_a.get(cv2.CAP_PROP_FRAME_HEIGHT))
    ww    = int(cap_a.get(cv2.CAP_PROP_FRAME_WIDTH))
    total = int(cap_a.get(cv2.CAP_PROP_FRAME_COUNT))
    cw    = int(ww * (2 - overlap_pct / 100))

    if offset > 0:  cap_b.set(cv2.CAP_PROP_POS_FRAMES, int(offset * fps))
    elif offset < 0: cap_a.set(cv2.CAP_PROP_POS_FRAMES, int(-offset * fps))

    fourcc      = cv2.VideoWriter_fourcc(*'mp4v')
    pano_writer = cv2.VideoWriter(pano_path, fourcc, fps, (cw, hh))

    write_status(jobs_dir, job_id, 2, 'Panorama Stitching', 50, 'Stitching all frames...')

    # Stage 4 - Per-frame stitch + detect
    print('  [4/7] Loading YOLO and EasyOCR...')
    write_status(jobs_dir, job_id, 3, 'Player & Ball Detection', 0, 'Loading YOLO...')
    model      = YOLO(model_path)
    ocr        = easyocr.Reader(['en'], gpu=True, verbose=False)
    kf         = make_kalman()
    kalman_ttl = int(kalman_secs * fps)
    kf_init    = False

    ball_positions = []
    ball_lost      = 0
    fidx           = 0
    last_st        = 0

    print('  [4/7] Processing frames...')
    while True:
        ret_a, fa = cap_a.read()
        ret_b, fb = cap_b.read()
        if not ret_a or not ret_b:
            break

        fa = undistort(fa, m1a, m2a)
        fb = undistort(fb, m1b, m2b)
        panorama = stitch_frame(fa, fb, H, overlap_pct, cw)

        ball_detected = False

        # ── Pass A: full-panorama inference for PLAYERS ───────────────────
        # Players are large enough to detect reliably at this scale.
        person_results = model(panorama, verbose=False, conf=0.35,
                               imgsz=1280, classes=[0])[0]

        for box in person_results.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            bcx = (x1 + x2) // 2

            ty1   = y1 + (y2-y1)//3
            ty2   = y1 + 2*(y2-y1)//3
            torso = panorama[ty1:ty2, x1:x2]

            shirt_num = None
            if torso.size > 100:
                t_up = cv2.resize(torso, None, fx=3, fy=3,
                                  interpolation=cv2.INTER_CUBIC)
                for txt in ocr.readtext(t_up, detail=0,
                                         allowlist='0123456789'):
                    try:
                        n = int(txt.strip())
                        if shirt_min <= n <= shirt_max:
                            shirt_num = n; break
                    except ValueError:
                        pass

            is_home = True
            if torso.size > 100:
                mb      = torso.reshape(-1, 3).mean(axis=0)
                is_home = (np.linalg.norm(mb - np.array(home_bgr)) <=
                           np.linalg.norm(mb - np.array(away_bgr)))

            tag_bgr = home_bgr if is_home else away_bgr
            squad   = squad_home if is_home else squad_away

            cv2.rectangle(panorama, (x1, y1), (x2, y2), tag_bgr, 2)

            parts = []
            if show_names and shirt_num and str(shirt_num) in squad:
                parts.append(squad[str(shirt_num)])
            if show_nums and shirt_num:
                parts.append(f'#{shirt_num}')
            elif not parts and shirt_num:
                parts.append(f'#{shirt_num}')
            if parts:
                draw_name_tag(panorama, bcx, y1, ' '.join(parts), tag_bgr)

        # ── Pass B: tiled inference for BALL ──────────────────────────────
        # The ball is tiny in a wide panorama (~4-8 px diameter).
        # Slicing into overlapping 640×640 tiles makes it appear ~4× larger
        # relative to the model input, dramatically improving detection.
        #
        # If Kalman has a recent prediction, we search only the region around
        # it (search_radius px) rather than the full panorama, saving ~60%
        # of inference time on frames where the ball has been tracked.
        ph, pw = panorama.shape[:2]
        TILE_SIZE   = 640
        TILE_STRIDE = 480   # 25% overlap between tiles
        BALL_CONF   = 0.25  # lower threshold — NMS handles false positives
        NMS_IOU     = 0.45

        # Determine search region
        if kf_init and ball_lost < kalman_ttl // 2:
            # Kalman predicted position — search a 1280×1280 window around it
            pred_x = int(np.clip(kf.x[0, 0], 0, pw))
            pred_y = int(np.clip(kf.x[1, 0], 0, ph))
            search_r = 640
            sx1 = max(0,  pred_x - search_r)
            sy1 = max(0,  pred_y - search_r)
            sx2 = min(pw, pred_x + search_r)
            sy2 = min(ph, pred_y + search_r)
        else:
            # No reliable prediction — search the whole panorama
            sx1, sy1, sx2, sy2 = 0, 0, pw, ph

        search_region = panorama[sy1:sy2, sx1:sx2]
        srh, srw = search_region.shape[:2]

        # Generate tile top-left corners within the search region
        tile_origins = []
        y = 0
        while y < srh:
            x = 0
            while x < srw:
                tile_origins.append((x, y))
                x += TILE_STRIDE
                if x + TILE_SIZE > srw and x < srw - 1:
                    x = max(0, srw - TILE_SIZE)  # last tile flush-right
                    tile_origins.append((x, y))
                    break
            y += TILE_STRIDE
            if y + TILE_SIZE > srh and y < srh - 1:
                y = max(0, srh - TILE_SIZE)
                continue

        # Collect all ball candidates across tiles (in panorama coords)
        raw_boxes  = []   # [x1, y1, x2, y2] in panorama coords
        raw_scores = []

        for (tx, ty) in tile_origins:
            tx2 = min(tx + TILE_SIZE, srw)
            ty2 = min(ty + TILE_SIZE, srh)
            tile = search_region[ty:ty2, tx:tx2]

            if tile.shape[0] < 32 or tile.shape[1] < 32:
                continue

            # Pad to square if tile is smaller than TILE_SIZE at edges
            padded = cv2.copyMakeBorder(
                tile,
                0, TILE_SIZE - tile.shape[0],
                0, TILE_SIZE - tile.shape[1],
                cv2.BORDER_CONSTANT, value=0
            ) if tile.shape[0] < TILE_SIZE or tile.shape[1] < TILE_SIZE else tile

            t_results = model(padded, verbose=False, conf=BALL_CONF,
                              imgsz=TILE_SIZE, classes=[32])[0]

            for tbox in t_results.boxes:
                score = float(tbox.conf[0])
                bx1, by1, bx2, by2 = map(float, tbox.xyxy[0])
                # Convert tile coords → search-region coords → panorama coords
                raw_boxes.append([
                    sx1 + tx + bx1,
                    sy1 + ty + by1,
                    sx1 + tx + bx2,
                    sy1 + ty + by2,
                ])
                raw_scores.append(score)

        # Non-maximum suppression to merge duplicate detections across tile overlaps
        if raw_boxes:
            boxes_np  = np.array(raw_boxes,  dtype=np.float32)
            scores_np = np.array(raw_scores, dtype=np.float32)
            keep = cv2.dnn.NMSBoxes(
                bboxes=boxes_np[:, :4].tolist(),
                scores=scores_np.tolist(),
                score_threshold=BALL_CONF,
                nms_threshold=NMS_IOU,
            )
            if len(keep) > 0:
                # Pick the highest-confidence surviving detection
                keep_flat = keep.flatten()
                best = keep_flat[np.argmax(scores_np[keep_flat])]
                bx1, by1, bx2, by2 = map(int, boxes_np[best])
                bcx = (bx1 + bx2) // 2
                bcy = (by1 + by2) // 2

                # Clamp to panorama bounds
                bcx = int(np.clip(bcx, 0, pw - 1))
                bcy = int(np.clip(bcy, 0, ph - 1))

                if not kf_init:
                    kf.x = np.array([[bcx],[bcy],[0],[0]], dtype=np.float32)
                    kf_init = True
                kf.predict()
                kf.update(np.array([[bcx],[bcy]], dtype=np.float32))
                ball_detected = True
                ball_lost     = 0
                ball_positions.append((fidx, bcx, bcy))
                cv2.circle(panorama, (bcx, bcy), 9, (0, 220, 255), 2)

        if not ball_detected and kf_init:
            kf.predict()
            ball_lost += 1
            if ball_lost < kalman_ttl:
                px, py = int(kf.x[0,0]), int(kf.x[1,0])
                ball_positions.append((fidx, px, py))
                cv2.circle(panorama, (px, py), 9, (0,220,255), 1)

        pano_writer.write(panorama)
        fidx += 1

        if fidx - last_st >= 300:
            pct = min(95, int(fidx/total*100)) if total else 50
            write_status(jobs_dir, job_id, 3, 'Player & Ball Detection', pct,
                         f'Frame {fidx}/{total}')
            last_st = fidx

    cap_a.release(); cap_b.release(); pano_writer.release()
    print(f'  Panorama: {fidx} frames written')

    # Stage 5 - Goal detection
    print('  [5/7] Goal detection...')
    write_status(jobs_dir, job_id, 4, 'Goal Event Detection', 20, 'Scanning...')
    goal_boxes = estimate_goal_boxes(cw, hh)
    goals      = detect_goals(ball_positions, goal_boxes, fps, goal_conf)
    print(f'  {len(goals)} goal(s) detected')
    write_status(jobs_dir, job_id, 4, 'Goal Event Detection', 100,
                 f'{len(goals)} goal(s) detected')

    # Stage 6 - Name tags (already rendered inline in stage 4)
    write_status(jobs_dir, job_id, 5, 'Name Tag Rendering', 100, 'Rendered inline')

    # Stage 7 - Ball-following render
    print('  [7/7] Rendering ball-following video...')
    render_ball_following(
        pano_path=pano_path, ball_positions=ball_positions,
        canvas_w=cw, h=hh, fps=fps, out_path=final_path,
        job=job, goals=goals,
        home_name=home_name, away_name=away_name,
        home_bgr=home_bgr, away_bgr=away_bgr,
        jobs_dir=jobs_dir, job_id=job_id,
    )

    if os.path.exists(pano_path):
        os.remove(pano_path)

    with open(os.path.join(output_dir, f'{job_id}_goals.json'), 'w') as gf:
        json.dump(goals, gf, indent=2)

    write_status(jobs_dir, job_id, 6, 'Video Render & Overlays', 100,
                 f'Done — {len(goals)} goal(s)', status='done',
                 extra={'output_file': final_path, 'goals': goals})

    print(f'  Complete -> {final_path}')
    return final_path


print('All pipeline functions loaded')


In [ ]:
# CELL 5 - Job watcher (runs continuously)
# Polls GameTracker/jobs/ every 30 seconds.
# When a job file appears, runs the full pipeline automatically.
# Leave this cell running. Submit jobs from the GameTracker web app.
import glob, traceback, torch

POLL_INTERVAL  = 30
processed_jobs = set()
gpu_name       = torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'

print(f'Watcher running on {gpu_name}')
print(f'Watching: {JOBS}')
print('The web app status indicator will turn green within 30 seconds.')
print()

while True:
    # Heartbeat - web app reads this to show green/amber status
    hb = {
        'alive':     True,
        'updated':   datetime.utcnow().isoformat(),
        'gpu':       gpu_name,
        'job_count': len(processed_jobs),
    }
    with open(os.path.join(JOBS, 'heartbeat.json'), 'w') as hf:
        json.dump(hb, hf)

    # File list - scan GameTracker/raw/ so web app can populate dropdowns
    VIDEO_EXTS = {'.mp4', '.mov', '.avi', '.mkv', '.mts', '.m2ts'}
    try:
        raw_files = []
        for fname in sorted(os.listdir(RAW)):
            if os.path.splitext(fname)[1].lower() in VIDEO_EXTS:
                fpath   = os.path.join(RAW, fname)
                size_mb = round(os.path.getsize(fpath) / (1024 * 1024), 1)
                raw_files.append({'name': fname, 'size_mb': size_mb})
        with open(os.path.join(JOBS, 'filelist.json'), 'w') as fl:
            json.dump({'files': raw_files,
                       'updated': datetime.utcnow().isoformat()}, fl)
    except Exception:
        pass  # non-fatal

    # Check for new jobs
    job_files = sorted(glob.glob(os.path.join(JOBS, 'job_*.json')))
    new_jobs  = [f for f in job_files
                 if os.path.basename(f) not in processed_jobs]

    if new_jobs:
        job_path = new_jobs[0]
        job_name = os.path.basename(job_path)

        with open(job_path) as jf:
            job = json.load(jf)

        job_id = job['job_id']
        print(f'\nJob received: {job_id}')
        print(f'  {job["match"]["home_name"]} vs {job["match"]["away_name"]}')

        try:
            run_job(
                job        = job,
                jobs_dir   = JOBS,
                output_dir = OUTPUT,
                work_dir   = WORK,
                model_path = os.path.join(MODELS, 'yolov8m.pt'),
            )
        except Exception as e:
            err = f'{type(e).__name__}: {e}'
            print(f'  Job failed: {err}')
            traceback.print_exc()
            write_error(JOBS, job_id, err)

        processed_jobs.add(job_name)

    else:
        t = datetime.now().strftime('%H:%M:%S')
        print(f'  [{t}] Waiting for jobs...', end='\r')

    time.sleep(POLL_INTERVAL)
